In [1]:
import numpy as np
import pandas as pd

In [2]:
movies = pd.read_csv('tmdb_5000_movies.csv')
credits = pd.read_csv('tmdb_5000_credits.csv')

In [3]:
movies.head(1)

,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count
0,237000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.avatarmovie.com/,19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2009-12-10,2787965087,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800


In [4]:
movies = movies.merge(credits, on='title')

In [5]:
movies = movies[['movie_id','title','overview','genres','keywords','cast']]

In [6]:
movies.head(1)

,movie_id,title,overview,genres,keywords,cast
0,19995,Avatar,"In the 22nd century, a paraplegic Marine is di...","[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...","[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...","[{""cast_id"": 242, ""character"": ""Jake Sully"", ""..."


In [7]:
movies.duplicated().sum()

0

In [8]:
movies.dropna(inplace=True)

In [9]:
import ast

In [10]:
def convert(text):
  l = []
  for i in ast.literal_eval(text):
    l.append(i['name'])
  return l

In [11]:
movies['genres'] = movies['genres'].apply(convert)

In [12]:
movies['keywords'] = movies['keywords'].apply(convert)

In [13]:
def convert3(text):
    L = []
    counter = 0
    for i in ast.literal_eval(text):
        if counter < 3:
            L.append(i['name'])
        counter+=1
    return L 

In [14]:
movies['cast'] = movies['cast'].apply(convert3)
movies.head()

,movie_id,title,overview,genres,keywords,cast
0,19995,Avatar,"In the 22nd century, a paraplegic Marine is di...","[Action, Adventure, Fantasy, Science Fiction]","[culture clash, future, space war, space colon...","[Sam Worthington, Zoe Saldana, Sigourney Weaver]"
1,285,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...","[Adventure, Fantasy, Action]","[ocean, drug abuse, exotic island, east india ...","[Johnny Depp, Orlando Bloom, Keira Knightley]"
2,206647,Spectre,A cryptic message from Bond’s past sends him o...,"[Action, Adventure, Crime]","[spy, based on novel, secret agent, sequel, mi...","[Daniel Craig, Christoph Waltz, Léa Seydoux]"
3,49026,The Dark Knight Rises,Following the death of District Attorney Harve...,"[Action, Crime, Drama, Thriller]","[dc comics, crime fighter, terrorist, secret i...","[Christian Bale, Michael Caine, Gary Oldman]"
4,49529,John Carter,"John Carter is a war-weary, former military ca...","[Action, Adventure, Science Fiction]","[based on novel, mars, medallion, space travel...","[Taylor Kitsch, Lynn Collins, Samantha Morton]"


In [15]:
def collapse(L):
    L1 = []
    for i in L:
        L1.append(i.replace(" ",""))
    return L1

In [16]:
movies['cast'] = movies['cast'].apply(collapse)
movies['genres'] = movies['genres'].apply(collapse)
movies['keywords'] = movies['keywords'].apply(collapse)

In [17]:
movies['overview'] = movies['overview'].apply(lambda x:x.split())

In [18]:
movies['tags'] = movies['cast'] + movies['genres'] + movies['keywords']

In [19]:
new = movies.drop(columns=['overview','genres','keywords','cast'])

In [20]:
new['tags']=new['tags'].apply(lambda x:" ".join(x))

In [21]:
new.head()

,movie_id,title,tags
0,19995,Avatar,SamWorthington ZoeSaldana SigourneyWeaver Acti...
1,285,Pirates of the Caribbean: At World's End,JohnnyDepp OrlandoBloom KeiraKnightley Adventu...
2,206647,Spectre,DanielCraig ChristophWaltz LéaSeydoux Action A...
3,49026,The Dark Knight Rises,ChristianBale MichaelCaine GaryOldman Action C...
4,49529,John Carter,TaylorKitsch LynnCollins SamanthaMorton Action...


In [22]:
pip install sklearn

Note: you may need to restart the kernel to use updated packages.


In [23]:
pip install --upgrade pip

Note: you may need to restart the kernel to use updated packages.


In [24]:
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(max_features=5000, stop_words='english')

In [25]:
vector=cv.fit_transform(new['tags'].ravel()).toarray()

In [26]:
vector.shape

(4806, 5000)

In [27]:
import nltk

In [28]:
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()

In [29]:
from nltk.stem.porter import PorterStemmer

def stem(text):
    ps = PorterStemmer()
    words = text.split()
    y = []
    for word in words:
        y.append(ps.stem(word))
    return " ".join(y)

In [30]:
new['tags']=new['tags'].apply(stem)

In [31]:
vector

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=int64)

In [32]:
from sklearn.metrics.pairwise import cosine_similarity

In [33]:
similarity=cosine_similarity(vector)

C:\Users\adity\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\utils\extmath.py:193: RuntimeWarning: invalid value encountered in matmul
  ret = a @ b


In [34]:
similarity

array([[1.        , 0.12838815, 0.11826248, ..., 0.06201737, 0.        ,
        0.        ],
       [0.12838815, 1.        , 0.13159034, ..., 0.        , 0.        ,
        0.        ],
       [0.11826248, 0.13159034, 1.        , ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.06201737, 0.        , 0.        , ..., 1.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 1.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        1.        ]])

In [35]:
def recommend(movie):
    index = new[new['title'] == movie].index[0]
    distances = sorted(list(enumerate(similarity[index])),reverse=True,key = lambda x: x[1])
    for i in distances[1:6]:
        print(new.iloc[i[0]].title)
        

In [36]:
recommend('Avatar')

Star Trek Into Darkness
The Lovers
Jupiter Ascending
Megaforce
The Time Machine


In [37]:
import pickle

In [38]:
pickle.dump(new.to_dict(), open('movie_dict.pkl','wb'))

In [39]:
pickle.dump(similarity,open('similarity.pkl','wb'))